<h1>Introduction</h1>

This notebook demonstrates how to scale a synthetic form of Full Waveform Inversion using multiple FPGAs. The FWI algorithm uses a Finite Difference Forward model and a Conjugate Gradient Descent optimization algorithm for the inversion. In the first step, we connect to an existing Dask cluster using it's scheduler's IP address.

In [ ]:
from dask.distributed import Client, progress, get_worker
import os
import binascii

#replace with IP address of the Dask scheduler
client = Client("tcp://131.180.106.138:8786")
client

<h1> Define experiment parameters</h1>

DIR_PATH => path to to the directory with the input/output folders  

XCLBIN_PATH_DEFAULT => Default path for the .xclbin file if one not provided via command line args  

DEVICE_NAME_DEFAULT => Default name for the FPGA device if one note provided via command line args  

In [3]:
DIR_PATH = "default/"
XCLBIN_PATH_DEFAULT="bitstreams/u280_xclbin/500_250_HBM/FullW.xclbin"
#DEVICE_NAME_DEFAULT="xilinx_u200_x"

In [6]:
ROW = 500
COL = 250

from pynq import Overlay, allocate, Device, lib
import numpy as np
import time


def init_hw_dotProduct(s_in_V,s2_in_V):       
    for col in range(COL):
        s2_in_V[col] = col * 1.0
        for row in range(ROW):
            s_in_V[row][col] = complex(row*1.0,row*col*0.33)
            
    return s_in_V, s2_in_V

def init_hw_updateDirection(resV_V,kappa_V):       
    for row in range(ROW):
        resV_V[row] = complex(row*1.0,row*0.33)
        for col in range(COL):
            kappa_V[row][col] = complex(row*1.0,row*col*0.33)
    return resV_V, kappa_V


def init_sw_dotProduct():
    s_in_SW = np.zeros(shape=(ROW,COL), dtype = np.complex64)
    s2_in_SW = np.zeros(shape=(COL,), dtype = np.float32)
    for col in range(COL):
        s2_in_SW[col] = col *1.0
        for row in range(ROW):        
            s_in_SW[row][col] = complex(row*1.0,row*col*0.33)            
    
    return s_in_SW, s2_in_SW

def init_sw_updateDirection():
    kappa_SW = np.zeros(shape=(ROW,COL), dtype=np.complex64)
    res_SW = np.zeros(shape=(ROW,),dtype=np.complex64)
        
    for row in range(ROW):
        res_SW[row] = complex(row*1.0, row*0.33)
        for col in range(COL):
            kappa_SW[row][col] = complex(row*1.0,row*col*0.33)
            
    return res_SW, kappa_SW

def dotProduct_SW(s_in, s2_in):
    out = np.zeros(ROW, dtype=np.complex64)
#     for row in range(ROW):
#         for col in range(COL):
#             out[row] +=s_in[row][col]*s2_in[col]
    for i in range(ROW):
        out[i] = np.dot(s_in[i],s2_in)
    return out 

def updateDirection_SW(res, kappa):
    out = np.zeros(shape=(COL),dtype = np.complex64)    
    
    for i in range(5):
        l_i = i * 10 * 10
        for j in range(10):
            l_j = j* 10
            for k in range(10):
                dummy = kappa[l_i + l_j + k]
                dummy.conjugate()
                out = out + res[l_i + l_j + k]*dummy.data
        
    return out      

devices = Device.devices
for i in devices:
    print(i.name)


# import and download the overlay to the PL.

ol = Overlay(XCLBIN_PATH_MULTCU, download=True, device=devices[0])

# set up the kernel IP's
dotprod = ol.dotprod_1
update = ol.update_1

dotprod2 = ol.dotp

# print(dotprod.signature)
# print(update.signature)

# print(dotprod)
# print(update)

print(ol.__doc__)


#Allocate memory for dotProduct buffers        
kappa_in_out_buffer = allocate(shape=(ROW,COL), dtype='c8', target=ol.HBM0)
vector_buffer = allocate(shape=(COL,), dtype=np.float32, target=ol.HBM1)

#Allocate memory for output buffers
out_buffer_dot = allocate(shape=(ROW,), dtype = np.complex64, target=ol.HBM2)

# print("allocated")
# print(ol.HBM0.mem_used) # 50 * 100 * 64 = 320 000
# print(ol.HBM1.mem_used) # 100 * 64 = 6400
# print(ol.HBM2.mem_used) # 50 * 64 = 3200

# kappa_in_out_buffer.sync_to_device()
# vector_buffer.sync_to_device()

# print(out_buffer_dot[0])
# dotprod.call(kappa_in_out_buffer,vector_buffer,out_buffer_dot)
# out_buffer_dot.sync_from_device()
# print(out_buffer_dot[0])

# print("executed")
# print(ol.HBM0.mem_used)
# print(ol.HBM1.mem_used)
# print(ol.HBM2.mem_used)


# %xdel kappa_in_out_buffer
# %xdel vector_buffer
# %xdel out_buffer_dot

# print("deleted")
# print(ol.HBM0.mem_used)
# print(ol.HBM1.mem_used)
# print(ol.HBM2.mem_used)

# Allocate memory for updateDirection buffers
res_V_buffer = allocate(shape=(ROW,),dtype=np.complex64, target=ol.HBM3)
kappa_V_buffer = allocate(shape=(ROW,COL), dtype= np.complex64, target=ol.HBM4)

out_buffer_upd = allocate(shape=(COL), dtype = np.complex64, target = ol.HBM5)

# res_V_buffer, kappa_V_buffer = init_hw_updateDirection(res_V_buffer,kappa_V_buffer)

# print("allocated")
# print(ol.HBM3.mem_used) # 50 * 64 = 3200
# print(ol.HBM4.mem_used) # 100 * 50 * 64 = 320 000
# print(ol.HBM5.mem_used) # 100 * 64 = 6400


# res_V_buffer.sync_to_device()
# kappa_V_buffer.sync_to_device()
# update.call(res_V_buffer,kappa_V_buffer,out_buffer_upd)
# out_buffer_upd.sync_from_device()

# print("executed")
# print(ol.HBM3.mem_used)
# print(ol.HBM4.mem_used)
# print(ol.HBM5.mem_used)



    
# print("deleted")
# print(ol.HBM3.mem_used)
# print(ol.HBM4.mem_used)
# print(ol.HBM5.mem_used)

# ol.free()


# print(ol.__doc__)



# print("allocated buffers")
# A = allocate(shape=(resolution,gridsize), dtype=np.complex64, target=ol.HBM0)
# B = allocate(shape=(gridsize,), dtype=np.float32, target=ol.HBM1)
# C = allocate(shape=(resolution,), dtype=np.complex64, target=ol.HBM2)

# D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=ol.HBM3)
# E = allocate(shape=(resolution,),dtype=np.complex64,  target=ol.HBM4)
# F = allocate(shape=(gridsize), dtype=np.complex64, target=ol.HBM5)

#fill the input buffers for dotProduct
kappa_in_out_buffer, vector_buffer = init_hw_dotProduct(kappa_in_out_buffer, vector_buffer)
kappa_dot_sw, vector_dot_sw = init_sw_dotProduct()

#fill the input buffer for updateDirection
res_V_buffer, kappa_V_buffer = init_hw_updateDirection(res_V_buffer,kappa_V_buffer)
res_upd_sw, kappa_upd_sw = init_sw_updateDirection()


#check if input buffers are the same for dotProduct
print(f"Inputs are equal for dotProduct? {(kappa_dot_sw==kappa_in_out_buffer).all()} and {(vector_dot_sw==vector_buffer).all()}")

#check if input buffers are the same for updateDirection
print(f"Inputs are equal for updateDirection? {(res_upd_sw==res_V_buffer).all()} and {(kappa_upd_sw==kappa_V_buffer).all()}")


########################
#####HARDWARE PART######
########################

#start the timer
start_time_hw = time.time()

kappa_in_out_buffer.sync_to_device()
vector_buffer.sync_to_device()
dotprod.call(kappa_in_out_buffer,vector_buffer,out_buffer_dot)
out_buffer_dot.sync_from_device()

# #perform accelerated dotProduct
# # dotProduct_HW(kappa_in_out_buffer,vector_buffer,out_buffer_dot)

# #dotProduct time
end_time_dot = time.time()-start_time_hw

res_V_buffer.sync_to_device()
kappa_V_buffer.sync_to_device()
update.call(res_V_buffer,kappa_V_buffer,out_buffer_upd)
out_buffer_upd.sync_from_device()



# #perform accelerated updateDirection

# #total hw time 
end_time_hw = time.time() - start_time_hw

#updateDirection time
end_time_upd = end_time_hw - end_time_dot

########################
#####SOFTWARE PART######
########################

#start the timer
start_time_sw = time.time()

#perform software dotProduct
out_dot_sw = dotProduct_SW(kappa_dot_sw, vector_dot_sw)

#dotProduct time
end_time_dot_sw = time.time()-start_time_sw

s = time.time()
#perform accelerated updateDirection
out_upd_sw = updateDirection_SW(res_upd_sw, kappa_upd_sw)

print(time.time()-s)
#total sw time
end_time_sw = time.time() - start_time_sw

#update Direction time
end_time_upd_sw = end_time_sw - end_time_dot_sw

equal_dot = (out_dot_sw.all()==out_buffer_dot.all())
equal_dot_close = np.allclose(out_dot_sw,out_buffer_dot,atol=0.1)

equal_upd = (out_upd_sw.all()==out_buffer_upd.all())
equal_upd_close = np.allclose(out_upd_sw,out_buffer_upd,atol=0.1)
            
#delete the buffers to avoid memory leaks
# del vector_dma
# del kappa_dot_dma
# del kappa_in_out_dma
# del resV_in_dma

#output text
print("-----------------------------------------------------")                          
print(f"Ouputs are equal for dotProduct? {equal_dot}")
print(f"Ouputs are equal for dotProduct within 0.1? {equal_dot_close}")
print("-----------------------------------------------------")             
print(f"Ouputs are equal for updateDirection? {equal_upd}")
print(f"Ouputs are equal for updateDirection within 0.1? {equal_upd_close}")
print("-----------------------------------------------------")                                               
print(f"Software dotProduct took:{end_time_dot_sw} ")
print(f"Hardware dotProduct took:{end_time_dot} ")                          
print(f"dotProduct speedup:{end_time_dot_sw/end_time_dot} ")            
print("-----------------------------------------------------")             
print(f"Software updateDirection took:{end_time_upd_sw} ")
print(f"Hardware updateDirection took:{end_time_upd} ")                          
print(f"updateDirection speedup:{end_time_upd_sw/end_time_upd} ")                  
print("-----------------------------------------------------")   
print(f"Total software version took: {end_time_sw}")
print(f"Total hardware version took: {end_time_hw}")
print(f"Total speed up: {end_time_sw/end_time_hw}")

%xdel kappa_in_out_buffer
%xdel vector_buffer
%xdel out_buffer_dot
%xdel kappa_V_buffer
%xdel res_V_buffer
%xdel out_buffer_upd
ol.free()

xilinx_u280_xdma_201920_3
(dotprod_in_matrix: 'void*', dotprod_in_vector: 'void*', dotprod_out: 'void*')
(update_input: 'void*', update_kappa: 'void*', update_output: 'void*')
Default documentation for overlay bitstreams/u280_xclbin/500_250_HBM/FullW.xclbin. The following
    attributes are available on this overlay:
    
    IP Blocks
    ----------
    dotprod_1            : pynq.overlay.DefaultIP
    dotprod_2            : pynq.overlay.DefaultIP
    update_1             : pynq.overlay.DefaultIP
    update_2             : pynq.overlay.DefaultIP
    
    Hierarchies
    -----------
    None
    
    Interrupts
    ----------
    None
    
    GPIO Outputs
    ------------
    None
    
    Memories
    ------------
    HBM0                 : Memory
    HBM1                 : Memory
    HBM2                 : Memory
    HBM3                 : Memory
    HBM4                 : Memory
    HBM5                 : Memory
    HBM6                 : Memory
    HBM7                 : Memory
  

<h1> Define the worker method </h1>

Here, we define the Python method which will be exeucuted on each of the Dask workers. This function calls the FWI implementation using the data partition it receives, and returns the output data (along with some performance statistics) to the caller (the Dask client)  


In [14]:

def run_on_worker(data, input_data, index):
    
    import numpy as np
    import json
    import os, sys
    import time
#     DIR_PATH = "default/"
#     XCLBIN_PATH_DEFAULT="FullW_u50_multhbm.xclbin"
#     DEVICE_NAME_DEFAULT="xilinx_u200_x"

    start_time = time.time()

    from FWI.grid2D import grid2D
    from FWI.sources import Sources
    from FWI.receivers import Receivers
    from FWI.frequenciesGroup import frequenciesGroup
    from FWI.finiteDifferenceForwardModel import FiniteDifferenceForwardModel
    from FWI.conjugateGradientInversion import ConjugateGradientInversion
    
    from pynq import Overlay, allocate, Device, lib
    
    devices = Device.devices
    for i in devices:
        print(i.name)
        

    # import and download the overlay to the PL.

    ol = Overlay(XCLBIN_PATH_DEFAULT, download=False, device=devices[0])

    # set up the kernel IP's
    dotprod = ol.dotprod_1
    update = ol.update_1

    resolution = 500
    gridsize = 500
    

    A = allocate(shape=(resolution,gridsize), dtype=np.complex64, target=ol.HBM0)
    B = allocate(shape=(gridsize,), dtype=np.float32, target=ol.HBM1)
    C = allocate(shape=(resolution,), dtype=np.complex64, target=ol.HBM2)
        
    D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=ol.HBM3)
    E = allocate(shape=(resolution,),dtype=np.complex64,  target=ol.HBM4)
    F = allocate(shape=(gridsize), dtype=np.complex64, target=ol.HBM5)
    
    print(f" import time {time.time() - start_time}")
    
    with open(DIR_PATH+"/input/GenericInput.json") as f:
        input_data = json.load(f)
    
    input_data["tolerance"] = 9.99*10**-7

    
    data = []
    with open(DIR_PATH+"/input/"+input_data["fileName"]+".txt") as f:
        for l in f:
            data.append(float(l))
    
    grid = grid2D([input_data["reservoirTopLeft"]["x"], input_data["reservoirTopLeft"]["z"]],
                  [input_data["reservoirBottomRight"]["x"], input_data["reservoirBottomRight"]["z"]],
                  [input_data["ngrid"]["x"], input_data["ngrid"]["z"]])
    source = Sources([input_data["sourcesTopLeft"]["x"], input_data["sourcesTopLeft"]["z"]],
                     [input_data["sourcesBottomRight"]["x"], input_data["sourcesBottomRight"]["z"]],
                     input_data["nSources"])
    receiver = Receivers([input_data["receiversTopLeft"]["x"], input_data["receiversTopLeft"]["z"]],
                         [input_data["receiversBottomRight"]["x"], input_data["receiversBottomRight"]["z"]],
                         input_data["nReceivers"])
    freq = frequenciesGroup(input_data["Freq"], input_data["c_0"])

    dot = []
    upd = []
    func_time = []
    rec_time = []
    chi = []
    low = 0
    high = 500

    print(f" len data: {len(data)}")

   
    for i in range(int(len(data)/500)):
        print(f"Iteration: {i}")
        model = FiniteDifferenceForwardModel(grid, source, receiver, freq, None, False, grid,resolution, dotprod, update, A,B,C)
#          
        inverse = ConjugateGradientInversion(None, model, input_data,D,E,F)
        input_data["max"] = 1000

        # Pre_processing
        referencePressureData = model.calculatePressureField(data[low:high])

        rec_time_s = time.time()
        chi.extend(inverse.reconstruct(referencePressureData, input_data).data)
        rec_time.append(time.time() - rec_time_s)

        dot.append(model.dot_time)
        upd.append(inverse.updtime)
        func_time.append(dot[i] + upd[i])
        low = high
        high += 500

    import math
    total_time = time.time() - start_time
    rec_time.append(sum(rec_time))
    dot.append(sum(dot))
    upd.append(sum(upd))
    func_time.append(sum(func_time))

    return {
#         "index": index,
#         "data":chi,
        "time": {"total": total_time,
            "dot":dot,
             "upd":upd,
             "func":func_time,
             "rec_time": rec_time
             }
    }

print(run_on_worker(1,1,1))


xilinx_u280_xdma_201920_3
 import time 0.0915989875793457
 len data: 500
Iteration: 0
Loop: 2 
Loop: 3 
Loop: 4 
Loop: 5 
Loop: 6 
Loop: 7 
Loop: 8 
Loop: 9 
Loop: 10 
Loop: 11 
Loop: 12 
Loop: 13 
Loop: 14 
Loop: 15 
Loop: 16 
Loop: 17 
Loop: 18 
Loop: 19 
Loop: 20 
Loop: 21 
Loop: 22 
Loop: 23 
Loop: 24 
Loop: 25 
Loop: 26 
Loop: 27 
Loop: 28 
Loop: 29 
Loop: 30 
Loop: 31 
Loop: 32 
Loop: 33 
Loop: 34 
Loop: 35 
Loop: 36 
Loop: 37 
Loop: 38 
Loop: 39 
Loop: 40 
Loop: 41 
Loop: 42 
Loop: 43 
Loop: 44 
Loop: 45 
Loop: 46 
Loop: 47 
Loop: 48 
Loop: 49 
Loop: 50 
Loop: 51 
Loop: 52 
Loop: 53 
Loop: 54 
Loop: 55 
Loop: 56 
Loop: 57 
{'time': {'total': 1.8799586296081543, 'dot': [0.14137864112854004, 0.14137864112854004], 'upd': [0.878199577331543, 0.878199577331543], 'func': [1.019578218460083, 1.019578218460083], 'rec_time': [1.6034753322601318, 1.6034753322601318]}}
